# Imports

In [1]:
from download_data import download_data

# Download data

In [2]:
data = download_data()    

In [6]:
y = data.iloc[:, -1]
x = data.iloc[:, :-1]

x.shape, y.shape

((958, 8), (958,))

# Weak Learner